In [72]:
import csv
import os
import tensorflow as tf
import keras
from keras import layers
import sklearn.model_selection
import numpy as np
import math

tf.config.set_visible_devices([], 'GPU')


In [73]:
u = csv.DictReader(open('u.csv', encoding='utf8'))
# print(u.fieldnames)

xs = []
for x in u:
    a = eval(x['Preferred Activities'])
    a = list(map(lambda x: x.strip(), a))
    d = eval(x['Bucket list destinations Sri Lanka'])
    d = list(map(lambda x: x.strip(), d))
    # print(a, d)
    xs.append((a, d))

# xs = xs[:10]

ks = list(set([x for a, d in xs for x in a]))
ks.sort()
vs = list(set([x for a, d in xs for x in d]))
vs.sort()
print(len(xs), len(ks), len(vs))


10000 68 157


In [74]:
p = csv.DictReader(open('p.csv',encoding='utf8'))
# print(u.fieldnames)

ps = {}
for x in p:
    n = x['name'].strip()
    if not (x['rating'] and x['user_ratings_total']):
        continue
    r = float(x['rating'])
    t = int(x['user_ratings_total'])
    t = math.log10(t)
    ps[n] = math.log(r*t)

ar = sum(ps.values())/len(ps)
print(len(ps),ar,min(ps.values()),max(ps.values()))

#print(vs)
#print(ps)
psx = np.array([ps[v] if v in ps else ar for v in vs])+1
#print(psx)


343 2.439213673429212 0.5899450208665175 3.056359854247776


In [75]:
model = keras.Sequential()
model.add(layers.InputLayer(input_shape=(len(ks),)))
model.add(layers.Dense(units=len(vs), use_bias=False, kernel_initializer='zeros'))
# model.add(layers.Dense(units=128))
# model.add(layers.Dense(units=128))
model.add(layers.Lambda(lambda x: x*psx))
# model.add(layers.Reshape(target_shape=(len(ks), 1)))
# model.add()

#model.compile(optimizer='adam', loss='mse')
#model.fit(gen(traind, 0), steps_per_epoch=len(traind), epochs=1000, callbacks=[tensorboard_callback])

model.summary()


Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 157)               10676     
                                                                 
 lambda_5 (Lambda)           (None, 157)               0         
                                                                 
Total params: 10,676
Trainable params: 10,676
Non-trainable params: 0
_________________________________________________________________


In [76]:
#model.set_weights([np.array([[0, 0, 0, 0, 0],[0, 0, 0, 0, 0,],[1, 0, 0, 0, 0]])])

In [77]:
ws = model.get_weights()[0]
ws[:, :] = 0
for a, d in xs:
    for aa in a:
        for dd in d:
            # print(a,aa,dd,ks.index(aa),vs.index(dd))
            ws[ks.index(aa)][vs.index(dd)] += 1

ws = sklearn.preprocessing.minmax_scale(ws, axis=1)

model.set_weights([ws])
print(model.get_weights())


[array([[0.00429185, 0.01287554, 0.02575107, ..., 0.05150215, 0.07725322,
        0.03433476],
       [0.02427185, 0.01456311, 0.02912621, ..., 0.05825243, 0.9223301 ,
        0.00970874],
       [0.00403226, 0.01612903, 0.01209677, ..., 0.03629032, 0.08870968,
        0.03225806],
       ...,
       [0.01578948, 0.01052632, 0.04736842, ..., 0.931579  , 1.        ,
        0.03157895],
       [0.02033898, 0.01016949, 0.02372881, ..., 0.04067796, 0.10508475,
        0.0440678 ],
       [0.02583026, 0.01107011, 0.01845019, ..., 0.05166052, 0.12546125,
        0.02214022]], dtype=float32)]


In [78]:
def f(i):
    # print(i)
    i = [1 if k in i else 0 for k in ks]
    o = model.call(inputs=tf.convert_to_tensor([i]))
    o = tf.math.top_k(o, k=5)
    # print(o.values.numpy().tolist()[0])
    o = o.indices.numpy().tolist()[0]
    o = [vs[i] for i in o]
    # print(o)
    return o


def g(z):
    a, b, c = z
    abc = f([a, b, c])
    # return abc
    for x in [c, b, a]:
        x = f([x])[0]
        if x not in abc:
            abc.insert(0, x)
    return abc[:5]


In [83]:
def t(x):
    if len(x) == 1:
        z = f(x)
    else:
        z = g(x)
    print(x, z)


t(['hot springs'])
t(['elephant rides'])
t(['museum visits'])
t(['waterfalls'])
t(['hot springs', 'elephant rides', 'waterfalls'])
t(['cycling', 'historical monuments', 'village homestays'])


['hot springs'] ['Madunagala Hot Water Spring', 'Kanniya Hot Springs', 'Mahapelessa Hot Springs', 'Maha Oya Hot Water Springs', 'Sigiriya']
['elephant rides'] ['Udawalawe', 'Pinnawala', 'Sigiriya', 'Mirissa Beach', 'Trincomalee']
['museum visits'] ['Colombo National Museum', 'Ratnapura Gem Museum', 'Kandy National Museum', 'National Museum Galle', 'Folk Museum']
['waterfalls'] ['Diyaluma Falls', "St Clair's Falls", 'Bambarakiri Ella', 'Bambarakanda Falls', 'Bopath Falls']
['hot springs', 'elephant rides', 'waterfalls'] ['Madunagala Hot Water Spring', 'Diyaluma Falls', 'Udawalawe', 'Pinnawala', 'Kanniya Hot Springs']
['cycling', 'historical monuments', 'village homestays'] ['Ella', 'Anuradhapura', 'Hatton', 'Haputale', 'Polonnaruwa']


In [80]:
import tqdm

zs = {}
for a, d in tqdm.tqdm(xs):
    v = len(set(g(a)).intersection(set(d)))
    if v not in zs:
        zs[v] = 0
    zs[v] += 1


100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:38<00:00, 261.86it/s]


In [81]:
zs

{5: 263, 3: 3307, 2: 3137, 4: 1641, 1: 1401, 0: 251}

In [82]:
sum([(k)*v for k, v in zs.items()])/sum(zs.values())/5*100

50.949999999999996